In [3]:
# 取り敢えずmnistで試す（CPU上での実行を想定しているため）
from chainer import Chain, Variable, optimizers, iterators, datasets, training, cuda
from chainer.training import extensions
from chainer.datasets import mnist
import chainer.functions as F
import chainer.links as L
import chainer

import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

C:\Users\suzuk\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
train, test = mnist.get_mnist(withlabel = False)
train = (train - 0.5) / 0.5
train = train.reshape(train.shape[0], 1, 28, 28)
batchsize = 128
dis_itr = iterators.SerialIterator(train, batchsize)
initializer = chainer.initializers.HeNormal()

In [5]:
class Generator(Chain):
    
    def __init__(self):
        super(Generator, self).__init__()
        with self.init_scope():
            self.fc1 = L.Linear(100, 1024, initialW = initializer)
            self.fc2 = L.Linear(1024, 128 * 7 * 7, initialW = initializer)
            self.dc3 = L.Deconvolution2D(128, 64, 4, 2, 1, initialW = initializer)
            self.dc4 = L.Deconvolution2D(64, 1, 4, 2, 1, initialW = initializer)
            
            self.bn1 = L.BatchNormalization(1024)
            self.bn2 = L.BatchNormalization(128 * 7 * 7)
            self.bn3 = L.BatchNormalization((64, 14, 14))
    def __call__(self, x):
        h = F.relu(self.bn1(self.fc1(x)))
        h = F.relu(self.bn2(self.fc2(h)))
        h = F.reshape(h, (h.shape[0], 128, 7, 7))
        h = F.relu(self.bn3(self.dc3(h)))
        h = F.tanh(self.dc4(h))
        return h

In [6]:
class Discriminator(Chain):
    
    def __init__(self):
        super(Discriminator, self).__init__()
        with self.init_scope():
            self.cv1 = L.Convolution2D(1, 64, 5, 2, initialW = initializer)
            self.cv2 = L.Convolution2D(64, 128, 5, 2, initialW = initializer)
            self.fc3 = L.Linear(None, 256, initialW = initializer)
            self.fc4 = L.Linear(256, 2, initialW = initializer)
            
            self.bn1 = L.BatchNormalization((64, 12, 12))
            self.bn2 = L.BatchNormalization((128, 4, 4))
            self.bn3 = L.BatchNormalization(256)
    def __call__(self, x):
        #h = F.leaky_relu(self.bn1(self.cv1(x)), 0.2)
        h = F.leaky_relu(self.cv1(x), 0.2)
        #h = F.leaky_relu(self.bn2(self.cv2(h)), 0.2)
        h = F.leaky_relu(self.cv2(h), 0.2)
        #h = F.leaky_relu(self.bn3(self.fc3(h)), 0.2)
        h = F.leaky_relu(self.fc3(h), 0.2)
        h = F.dropout(h, 0.5)
        h = self.fc4(h)
        return h

In [7]:
max_epoch = 50

GENERATED_IMAGE_PATH = 'generated_images/'
if not os.path.exists(GENERATED_IMAGE_PATH):
    os.mkdir(GENERATED_IMAGE_PATH)

In [8]:
def train():
    chainer.cuda.memory_pool.free_all_blocks()
    
    model_gen = Generator()
    model_dis = Discriminator()
    
    opt_gen = optimizers.Adam(alpha = 2e-4, beta1 = 0.5)
    opt_dis = optimizers.Adam(alpha = 2e-4, beta1 = 0.5)
    opt_gen.setup(model_gen)
    opt_dis.setup(model_dis)
    opt_gen.add_hook(chainer.optimizer.WeightDecay(0.00001))
    opt_dis.add_hook(chainer.optimizer.WeightDecay(0.00001))
    
    gpu_device = 0
    cuda.get_device(gpu_device).use()
    model_gen.to_gpu(gpu_device)
    model_dis.to_gpu(gpu_device)
    xp = cuda.cupy
    
    dis_itr.epoch = 0
    while dis_itr.epoch < max_epoch:
        real_images = xp.array(dis_itr.next())
        y = model_dis(real_images)
        loss_dis = F.softmax_cross_entropy(y, Variable(xp.zeros(batchsize).astype(np.int32)))
        
        noise = xp.array(np.random.uniform(-1, 1, (batchsize, 100)).astype(np.float32))
        generated_images = model_gen(noise)
        y = model_dis(generated_images)
        
        loss_dis += F.softmax_cross_entropy(y, Variable(xp.ones(batchsize).astype(np.int32)))
        model_dis.cleargrads()
        loss_dis.backward()
        opt_dis.update()
        
        noise = xp.array(np.random.uniform(-1, 1, (batchsize, 100)).astype(np.float32))
        y = model_dis(model_gen(noise))
        loss_gen = F.softmax_cross_entropy(y, Variable(xp.zeros(batchsize).astype(np.int32)))
        model_gen.cleargrads()
        loss_gen.backward()
        opt_gen.update()
        
        if dis_itr.is_new_epoch:
            print("epoch:", dis_itr.epoch)
            print("loss_dis:", loss_dis, "  loss_gen:", loss_gen)
            for n,image in enumerate(generated_images[:10]):
                image = cuda.to_cpu(image.data)*127.5 + 127.5
                Image.fromarray(image.astype(np.uint8).reshape(28, 28)) \
                                                      .save(GENERATED_IMAGE_PATH+"%03d_%03d.png" % (dis_itr.epoch, n))

In [ ]:
train()

epoch: 1
loss_dis: variable(0.13320386)   loss_gen: variable(5.2728815)
epoch: 2
loss_dis: variable(0.17901576)   loss_gen: variable(4.3863845)
epoch: 3
loss_dis: variable(0.3040013)   loss_gen: variable(3.4799953)
epoch: 4
loss_dis: variable(0.31982332)   loss_gen: variable(2.9799676)
epoch: 5
loss_dis: variable(0.9771055)   loss_gen: variable(0.9219672)
epoch: 6
loss_dis: variable(0.61661637)   loss_gen: variable(2.4262328)
epoch: 7
loss_dis: variable(0.6992971)   loss_gen: variable(2.583445)
epoch: 8
loss_dis: variable(0.65144277)   loss_gen: variable(2.6459694)
epoch: 9
loss_dis: variable(0.9196221)   loss_gen: variable(1.3410692)
epoch: 10
loss_dis: variable(0.805462)   loss_gen: variable(1.9157201)
epoch: 11
loss_dis: variable(0.704929)   loss_gen: variable(1.7037642)
epoch: 12
loss_dis: variable(0.9544689)   loss_gen: variable(1.9467893)
epoch: 13
loss_dis: variable(0.87063134)   loss_gen: variable(1.664388)
epoch: 14
loss_dis: variable(0.8301674)   loss_gen: variable(1.7489468)